In [66]:
# Part 1:
import pandas as pd

# Load the dataset. Check the first 5 rows.  
crime = pd.read_csv("crime_data.csv")

print("Original Dataset:")
print(crime.head())

# Identify columns with missing values and their respective counts. Drop columns where more than 50% of the data is missing (store this version as a new dataset). 
missing_values = crime.isnull().sum()

threshold = len(crime) * 0.5 # If 50% is empty
crime_cleaned = crime.dropna(thresh=threshold, axis=1).copy()  # Create a copy

crime_cleaned.to_csv("crime_dataset_cleaned.csv", index=False) 
print("\nCleaned Dataset (First 5 Rows):")
print(crime_cleaned.head())

# Convert the DATE OCC column to a datetime format. Extract the year, month, and day into separate columns. Create a new column for the hour using the TIME OCC column. 
crime_cleaned['DATE OCC'] = pd.to_datetime(crime_cleaned['DATE OCC'], format='%d/%m/%Y %I:%M:%S %p', errors='coerce')

# Extract separate columns
crime_cleaned['Year'] = crime_cleaned['DATE OCC'].dt.year
crime_cleaned['Month'] = crime_cleaned['DATE OCC'].dt.month
crime_cleaned['Day'] = crime_cleaned['DATE OCC'].dt.day

# Create column for hour
crime_cleaned['Hour'] = crime_cleaned['TIME OCC'].apply(lambda x: str(x).zfill(4)[:2])

crime_cleaned.to_csv("crime_dataset_cleaned.csv", index=False)
print("\nUpdated Dataset with Date and Time Extracted (First 5 Rows):")
print(crime_cleaned.head())

# Filter the dataset for crimes that occurred in 2023. Further filter crimes with the description BURGLARY in the Crm Cd Desc column. 
crime_2023 = crime_cleaned[crime_cleaned['Year'] == 2023]

burglary_2023 = crime_2023[crime_2023['Crm Cd Desc'] == 'BURGLARY']

burglary_2023.to_csv("burglary_2023.csv", index=False)

# Group the data by AREA NAME and calculate the total number of crimes and the average victim age. Sort the results by total crimes in descending order.
area_stats = crime_cleaned.groupby('AREA NAME').agg(
    Total_Crimes=('DR_NO', 'count'),
    Avg_Victim_Age=('Vict Age', 'mean')).reset_index()

# Sorted data
area_stats_sorted = area_stats.sort_values(by='Total_Crimes', ascending=False)

area_stats_sorted.to_csv("area_stats.csv", index=False)
print("\nSorted Dataset:")
print(area_stats_sorted.head())



Original Dataset:
       DR_NO               Date Rptd                DATE OCC  TIME OCC  AREA  \
0  241711715  08/01/2024 12:00:00 AM  08/01/2024 12:00:00 AM      1319    17   
1  231014031  09/21/2023 12:00:00 AM  09/15/2023 12:00:00 AM      1930    10   
2  231010808  06/27/2023 12:00:00 AM  06/26/2023 12:00:00 AM      1230    10   
3  211410441  04/25/2021 12:00:00 AM  04/25/2021 12:00:00 AM      2330    14   
4  211114569  10/25/2021 12:00:00 AM  10/25/2021 12:00:00 AM      1455    11   

     AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0   Devonshire         1791         1     440   
1  West Valley         1011         2     354   
2  West Valley         1015         2     354   
3      Pacific         1488         2     626   
4    Northeast         1123         1     210   

                          Crm Cd Desc  ... Status  Status Desc Crm Cd 1  \
0  THEFT PLAIN - PETTY ($950 & UNDER)  ...     IC  Invest Cont    440.0   
1                   THEFT OF IDENTITY  ...     IC  Inves

In [63]:
# Part 2
# Find the top 3 most frequent Crm Cd Desc values
top_crimes = crime_cleaned['Crm Cd Desc'].value_counts().head(3)
print("Top 3 Most Frequent Crime Descriptions:")
print(top_crimes)

# Group the data by Hour and count the number of crimes
hourly_crimes = crime_cleaned.groupby('Hour').size().reset_index(name='Crime_Count')
print("\nNumber of Crimes by Hour:")
print(hourly_crimes)

# Group the data by Vict Sex and calculate total crimes and average victim age
victim_stats = crime_cleaned.groupby('Vict Sex').agg(
    Total_Crimes=('DR_NO', 'count'),
    Avg_Victim_Age=('Vict Age', 'mean')).reset_index()
print("\nCrime Statistics by Victim Sex:")
print(victim_stats)

Top 3 Most Frequent Crime Descriptions:
Crm Cd Desc
VEHICLE - STOLEN            5733
BATTERY - SIMPLE ASSAULT    3715
THEFT OF IDENTITY           3169
Name: count, dtype: int64

Number of Crimes by Hour:
   Hour  Crime_Count
0    00         2025
1    01         1490
2    02         1241
3    03         1063
4    04          975
5    05          848
6    06         1106
7    07         1302
8    08         1776
9    09         1841
10   10         2110
11   11         2114
12   12         3446
13   13         2246
14   14         2504
15   15         2667
16   16         2597
17   17         2914
18   18         3015
19   19         2762
20   20         2830
21   21         2553
22   22         2469
23   23         2106

Crime Statistics by Victim Sex:
  Vict Sex  Total_Crimes  Avg_Victim_Age
0        F         17922       38.164156
1        H             3       36.333333
2        M         20076       37.165621
3        X          4836        2.672043


In [58]:
# Part 4: 
# Create a new column, Severity Score / Assign 1 point for all crimes
crime['Severity Score'] = 1

# Assign 3 points for crimes under BUGLARY
crime.loc[crime['Crm Cd Desc'].str.contains("BURGLARY", case=False, na=False), 'Severity Score'] += 2 

# Assign 5 points if the weapon was used 
crime.loc[crime['Weapon Used Cd'].notnull(), 'Severity Score'] += 4 

severity_by_area = crime.groupby('AREA NAME')['Severity Score'].sum().reset_index()

# Sort by total severity score in descending order
severity_by_area = severity_by_area.sort_values(by='Severity Score', ascending=False)

# Display results
print("Severity by Area: ")
print(severity_by_area)


Severity by Area: 
      AREA NAME  Severity Score
1       Central            9603
0   77th Street            9479
14    Southeast            7321
15    Southwest            7265
9        Newton            7135
6     Hollywood            7084
12      Pacific            7012
11      Olympic            6619
13      Rampart            6526
8   N Hollywood            5823
20     Wilshire            5583
19  West Valley            5324
18      West LA            5131
4        Harbor            5104
17     Van Nuys            5050
10    Northeast            4811
2    Devonshire            4771
16      Topanga            4768
7       Mission            4671
5    Hollenbeck            4629
3      Foothill            3999
